In [1]:

from typing import Dict, Any
import socketio
import asyncio
import threading
from uuid_extensions import uuid7str

In [2]:
class DataNamespace(socketio.AsyncClientNamespace):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sent_messages = {}

    # ------------------------------ Event Handlers ------------------------------

    def on_connect(self):
        print("EVENT [connect]: Connected to the data server.")

    def on_disconnect(self):
        print("EVENT [disconnect]: Disconnected from the data server.")

    # ------------------------------ JSONRPC Method Handlers ------------------------------

    def on_message(self, data):
        print(f"EVENT [message]: Received message: {data}")
        msg_id = data['id']
        result = data['result'] if 'result' in data else None
        error = data['error'] if 'error' in data else None
        jsonrpc = data['jsonrpc']
        if not jsonrpc == '2.0':
            raise Exception('Invalid JSON-RPC version received from server')
        # handle response based on sent messages 
        if not msg_id in self.sent_messages:
            raise Exception(f"Received message with id: {msg_id} that was never sent")
        ori_msg = self.sent_messages[msg_id]
        # handle errors 
        if error:
            print(f"For request {msg_id} received error: {error}")
            return
        # handle results
        if ori_msg['method'] == 'hg_getMarkets':
            print(f"Successfully got markets {result}")
        else: 
            raise Exception(f"Unknown method {ori_msg['method']} in sent message {msg_id}")

sio = socketio.AsyncClient()
NAMESPACE = '/data'
SERVER_URL = f'wss://api-origin-staging-v2.hourglass.com/{NAMESPACE}'

ns = DataNamespace(NAMESPACE)
sio.register_namespace(ns)

loop = asyncio.new_event_loop()  # Create a new event loop
asyncio.set_event_loop(loop)  # Set the new loop as the current event loop

# Connect to the server
asyncio.run_coroutine_threadsafe(sio.connect(SERVER_URL, namespaces=[NAMESPACE], transports=['websocket']), loop)

# Start the event loop in a separate thread
t = threading.Thread(target=loop.run_forever)
t.start()

EVENT [connect]: Connected to the data server.
EVENT [message]: Received message: {'jsonrpc': '2.0', 'result': {'markets': [{'id': 1, 'name': 'wstETH-weETH', 'description': 'wstETH to weETH market pair', 'rfqTtlMsecs': 600000, 'defaultOrderTtlSecs': 300, 'minOrderTtlSecs': 120, 'maxOrderTtlSecs': 600, 'feeBps': 10, 'asset0Id': 2, 'asset1Id': 3, 'marketStatus': 'ONLINE', 'asset0': {'id': 2, 'symbol': 'wstETH', 'tokenType': 'ERC20', 'erc20Id': 2, 'erc721Id': None, 'erc1155Id': None, 'info': {'assetId': 2, 'address': '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', 'chain': 'Ethereum'}, 'erc20': {'id': 2, 'chain': 'Ethereum', 'address': '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', 'name': 'Wrapped liquid staked Ether 2.0', 'symbol': 'wstETH', 'description': None, 'tokenDecimals': 18}}, 'asset1': {'id': 3, 'symbol': 'weETH', 'tokenType': 'ERC20', 'erc20Id': 3, 'erc721Id': None, 'erc1155Id': None, 'info': {'assetId': 3, 'address': '0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee', 'chain': 'Ethere

In [3]:
async def emit_message(method: str, params: Dict[str, Any]): 
    msg_id = uuid7str()
    msg = {
        'jsonrpc': '2.0',
        'method': method,
        'params': params,
        'id': msg_id
    }
    print(f"Sending message: {msg}")
    ns.sent_messages[msg_id] = msg
    res = await sio.emit('message', msg, namespace=NAMESPACE)
    return res 

async def get_markets():
    method = 'hg_getMarkets'
    params = {}
    res = await emit_message(method, params)
    print(f"Attempted to get markets")
    print(res)

In [8]:
# DISCONNECT 
asyncio.run_coroutine_threadsafe(sio.disconnect(), loop) 

EVENT [disconnect]: Disconnected from the RFQ server.


<Future at 0x120f054d0 state=pending>

In [4]:
# GET MARKETS
asyncio.run_coroutine_threadsafe(get_markets(), loop)

Sending message: {'jsonrpc': '2.0', 'method': 'hg_getMarkets', 'params': {}, 'id': '06619bf1-aff7-77c0-8000-8a0ed8bde83d'}
Attempted to get markets
None


<Future at 0x1187b6790 state=pending>